In [29]:
import pandas as pd
!pip install python-docx
import bibtexparser
from docx import Document
from scholarly import scholarly
def parse_bibtex(file_path):
    with open(file_path, 'r', encoding='utf-8') as bibtex_file:
        bib_database = bibtexparser.load(bibtex_file)

    publications = []
    for entry in bib_database.entries:
        publications.append({
            "Author": entry.get("author", "Unknown"),
            "Title": entry.get("title", "Unknown"),
            "Year": entry.get("year", "Unknown"),
            "Journal": entry.get("journal", entry.get("booktitle", "Conference"))
        })
    return pd.DataFrame(publications)


def load_excel(file_path):
    return pd.read_excel(file_path)
def crawl_scholar(authors):
    publications = []
    for name in authors:
        try:
            search_query = scholarly.search_author(name)
            author = scholarly.fill(next(search_query))
            for pub in author.get('publications', [])[:5]:
                filled_pub = scholarly.fill(pub)
                publications.append({
                    "Author": name,
                    "Title": filled_pub.get("bib", {}).get("title", "Unknown"),
                    "Year": filled_pub.get("bib", {}).get("pub_year", "Unknown"),
                    "Journal": filled_pub.get("bib", {}).get("venue", "Unknown")
                })
        except Exception as e:
            print(f"Error fetching data for {name}: {e}")
    return pd.DataFrame(publications)

def filter_by_year(df, start_year, end_year):
    df = df.copy()
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
    df = df.dropna(subset=['Year'])
    return df[(df['Year'] >= start_year) & (df['Year'] <= end_year)]

def export_to_word(df, file_name):
    doc = Document()
    doc.add_heading('Publication Summary', level=1)

    for year in sorted(df['Year'].dropna().unique()):
        doc.add_heading(f'Year {int(year)}', level=2)
        year_data = df[df['Year'] == year]
        for _, row in year_data.iterrows():
            journal_value = row.get('Journal', row.get('journal', 'Unknown'))
            doc.add_paragraph(f"Title: {row['Title']}\nAuthor(s): {row['Author']}\nJournal/Conference: {journal_value}")

    doc.save(file_name)

def export_to_excel(df, file_name):
    df.to_excel(file_name, index=False)

def generate_summary_from_sample(sample_file="sample_publications.xlsx", start_year=2015, end_year=2025):
    df = load_excel(sample_file)
    filtered_data = filter_by_year(df, start_year, end_year)
    export_to_word(filtered_data, "sample_publication_summary.docx")
    export_to_excel(filtered_data, "sample_publication_summary.xlsx")
    print("Sample publication summaries generated successfully!")

if __name__ == "__main__":
    generate_summary_from_sample()


Sample publication summaries generated successfully!
